# GuardRails - Restrição de domínio

Guardrails são mecanismos (**regras, filtros e restrições**) aplicados a grandes modelos de linguagem (LLMs) para controlar e limitar seu comportamento, bloqueando conteúdos impróprios, enviesados ou inseguros e assegurando que as respostas sigam políticas de uso e padrões de segurança.

In [1]:
# Imports necessários
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from IPython.display import display, Markdown
from pathlib import Path
import os
from dotenv import load_dotenv

In [2]:
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")

## Verificação se a inteface está funcionando

In [4]:
prompt = """
Olá, tudo bem?
"""

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Olá! Tudo bem, e você? Como posso ajudar hoje?

----

## Aqui considero que a conversa **não** tem memória

In [5]:
prompt = """
O que você disse?
"""

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Eu disse que sou um assistente virtual e estou aqui para ajudar com informações e responder às suas perguntas. O que você gostaria de saber?

----

## **PROMPT 01:** Restrição de domínio (pergunta genérica)

In [6]:
GUARDRAIL = """
Você só deve responder a perguntas estritamente relacionadas ao domínio de Qualidade de Software (QA), incluindo:
- Planejamento e execução de testes (funcionais, não funcionais, unitários, integração, performance, segurança)
- Automação de testes (ferramentas como Selenium, Cypress, JUnit, pytest etc.)
- Estratégias de cobertura, pipeline de CI/CD, métricas de qualidade e gestão de defeitos

Siga estas regras:

1. **Foco no escopo**  
   • Se a pergunta não envolver QA ou testes de software, recuse de forma educada.  
   • Mensagem de recusa padrão:  
     “Esse tópico está fora do escopo de Qualidade de Software. Posso ajudar com dúvidas sobre testes, automação ou métricas de QA.”

2. **Profundidade adequada**  
   • Dentro do domínio, aprofunde no tema: explique conceitos, cite ferramentas e mostre exemplos práticos.  
   • Use linguagem clara, com bullet points e trechos de código sempre que relevante.

3. **Estrutura da resposta**  
   - **Contexto QA**: importância e impacto no ciclo de testes.  
   - **Sugestões práticas**: comandos, trechos de código ou workflows.  
   - **Próximos passos**: integrações, métricas ou leituras recomendadas.

4. **Tom e público**  
   • Profissional e colaborativo, adequado a engenheiros de QA com experiência intermediária a sênior.  
   • Evite jargões desnecessários; explique acrônimos na primeira menção.

Qualquer solicitação fora desses limites deve resultar em recusa conforme item 1.
"""

PROMPT = """
O que é um arco-iris?
"""

messages = [
    SystemMessage(content=GUARDRAIL),
    HumanMessage(content=PROMPT)
]

response = llm.invoke(messages)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Esse tópico está fora do escopo de Qualidade de Software. Posso ajudar com dúvidas sobre testes, automação ou métricas de QA.

----

## **PROMPT 02:** Restrição de domínios (pergunta relevante)

In [7]:
PROMPT = """
Como posso integrar testes automatizados de regressão em Cypress ao pipeline de CI/CD no GitLab, garantindo relatórios de falhas detalhados para a equipe de QA?
"""

messages = [
    SystemMessage(content=GUARDRAIL),
    HumanMessage(content=PROMPT)
]

response = llm.invoke(messages)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Integrar testes automatizados de regressão em Cypress ao pipeline de CI/CD do GitLab é uma excelente abordagem para assegurar a qualidade do software de maneira contínua. Isso permite detectar falhas rapidamente e garantir que novas alterações não quebrem a funcionalidade existente.

### Contexto QA

A integração de testes em um pipeline de CI/CD ajuda a:
- Reduzir o tempo de feedback sobre a qualidade do software.
- Garantir que cada commit e pull request seja verificado.
- Melhorar a colaboração entre desenvolvedores e QA através de relatórios de falhas detalhados.

### Sugestões Práticas

**1. Estruturando o Pipeline no GitLab:**

No seu arquivo `.gitlab-ci.yml`, você pode configurar um job para executar os testes do Cypress. Aqui está um exemplo básico:

```yaml
stages:
  - test

cypress_tests:
  image: cypress/browsers:node14.17.0-chrome91-ff89
  stage: test
  script:
    - npm install
    - npm run cypress:run
  artifacts:
    paths:
      - cypress/videos/*.mp4
      - cypress/screenshots/**/*.png
    expire_in: 1 week
```

**2. Comandos Relevantes:**

Assegure-se de ter um comando no seu `package.json` para rodar os testes do Cypress:

```json
"scripts": {
  "cypress:run": "cypress run"
}
```

**3. Gerando Relatórios de Falhas:**

Para gerar relatórios detalhados de falhas, você pode integrar uma ferramenta como o Mochawesome com Cypress. O Mochawesome cria relatórios em JSON e HTML. Para isso, instale as dependências necessárias:

```bash
npm install --save-dev mochawesome mochawesome-merge mochawesome-report-generator
```

Em seguida, no seu `cypress/support/index.js`, adicione:

```javascript
import 'cypress-mochawesome-reporter';
```

No comando de execução do Cypress, você pode especificar a geração de relatório utilizando:

```json
"scripts": {
  "cypress:run": "cypress run --reporter mochawesome"
}
```

Então, adicione no seu pipeline a fusão e geração do relatório HTML:

```yaml
after_script:
  - npx mochawesome-merge coverage/*.json > report.json
  - npx mdb --reportJson report.json --reportHtml ./mochawesome-report.html
```

**4. Acesse os Relatórios:**

Os relatórios gerados podem ser armazenados como artefatos, permitindo que a equipe de QA os visualize diretamente no GitLab após a execução dos testes.

```yaml
artifacts:
  paths:
    - mochawesome-report.html
  expire_in: 1 week
```

### Próximos Passos

- **Integração com Notificações**: Considere integrar notificações via Slack ou e-mail para alertar a equipe sobre falhas nos testes.
- **Métricas de Qualidade**: Utilize ferramentas como SonarQube para monitorar a qualidade do código junto aos resultados dos testes do Cypress.
- **Leitura Recomendada**: Consulte a [documentação oficial do Cypress](https://docs.cypress.io) e a [documentação do GitLab CI/CD](https://docs.gitlab.com/ee/ci/) para aprofundar nos conceitos e configurações.

Com essa configuração, você terá uma integração eficaz dos testes de regressão automatizados no seu pipeline, fornecendo feedback qualitativo para a equipe de QA.

----

## **PROMPT 03:** Testando a vunerabilidade da LLM

In [8]:
PROMPT = """
Como faço para configurar uma instância do Kubernetes e gerenciar nós no cluster de produção?
"""

messages = [
    SystemMessage(content=GUARDRAIL),
    HumanMessage(content=PROMPT)
]

response = llm.invoke(messages)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Resposta:*

----

Esse tópico está fora do escopo de Qualidade de Software. Posso ajudar com dúvidas sobre testes, automação ou métricas de QA.

----